# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [2]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [4]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    vertexai=True,
    temperature=0
)

# Create a moltbook account for your agent

In [5]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [6]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155253144)

'69681639'

In [13]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \https://www.moltbook.com/claim/moltbook_claim_qR_HaUHzEFO13OF3uI3gPyQZWDhgcWgI \
  -d '{"name": "69681639", "description": "Va"}'

{"statusCode":409,"message":"Agent name already taken","timestamp":"2026-02-26T03:51:35.977Z","path":"/api/v1/agents/register","error":"Conflict"}<!DOCTYPE html><html lang="en"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1"/><link rel="stylesheet" href="/_next/static/chunks/b35d7cbd7f232d6a.css" data-precedence="next"/><link rel="stylesheet" href="/_next/static/chunks/0d70d4a0e992f9b7.css" data-precedence="next"/><link rel="preload" as="script" fetchPriority="low" href="/_next/static/chunks/102a67e358ac219f.js"/><script src="/_next/static/chunks/797fcb0710da3be2.js" async=""></script><script src="/_next/static/chunks/2013d09b579e2150.js" async=""></script><script src="/_next/static/chunks/785528a67640bd6e.js" async=""></script><script src="/_next/static/chunks/415a6dfb943bcc8e.js" async=""></script><script src="/_next/static/chunks/2375e3fd2a9b8844.js" async=""></script><script src="/_next/static/chunks/turbopack-d70fc767af48bb16.js" as

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

# A simple agent to interact with moltbook

In [15]:
# Create a tool set to interact with moltbook (Strictly follow https://www.moltbook.com/skill.md)
import os
import requests
from langchain_core.tools import tool
from google.colab import userdata

# -------------------------- basic --------------------------
MOLTBOOK_API_KEY = userdata.get('MOLTBOOK')
BASE_URL = "https://www.moltbook.com/api/v1"
HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json",
    "User-Agent": "Moltbook-Agent/1.0"
}

# -------------------------- 1. authentication --------------------------
@tool
def moltbook_verify_auth() -> dict:
    """
    Verify Moltbook API key validity (Official valid API: /api/v1/me)
    Returns:
        dict: Authentication result with success status and official response
    """
    try:
        response = requests.get(
            url=f"{BASE_URL}/me",
            headers=HEADERS,
            timeout=15,
            allow_redirects=False
        )

        raw_content = response.text.strip()
        print(f"[DEBUG] Auth Response Status: {response.status_code}")
        print(f"[DEBUG] Auth Raw Content: {raw_content[:200]}")

        if not raw_content:
            if response.status_code == 200:
                result = {
                    "success": True,
                    "status_code": 200,
                    "message": "API authentication successful (empty valid response)",
                    "data": {}
                }
            else:
                result = {
                    "success": False,
                    "status_code": response.status_code,
                    "error": f"Authentication failed (status {response.status_code}), empty response",
                    "hint": "Check if API Key is valid (must start with moltbook_)"
                }
        elif raw_content.startswith("{"):
            result = {
                "success": True,
                "status_code": response.status_code,
                "message": "API authentication successful (Moltbook official verify)",
                "data": response.json()
            }
        else:
            result = {
                "success": False,
                "status_code": response.status_code,
                "error": f"Authentication failed: Non-JSON response ({raw_content[:100]})",
                "hint": "API Key invalid or endpoint error"
            }
        return result

    except requests.exceptions.HTTPError as e:
        return {
            "success": False,
            "status_code": response.status_code if 'response' in locals() else None,
            "error": f"Authentication failed (HTTP Error): {str(e)}",
            "hint": "Check if your MOLTBOOK API key is valid (refer to skill.md #Auth)"
        }
    except Exception as e:
        return {
            "success": False,
            "error": f"Authentication failed (System Error): {str(e)}",
            "hint": "Check network or API base URL"
        }

# -------------------------- 2. subscribe_submolt --------------------------
@tool
def subscribe_submolt(submolt_name: str = "ftec5660") -> dict:
    """
    Subscribe to a Moltbook submolt (Official API: /submolts/{name}/subscribe)
    Args:
        submolt_name: Name of the submolt (default: ftec5660, match homework requirement)
    Returns:
        dict: Subscription result with success status
    """
    try:
        response = requests.post(
            url=f"{BASE_URL}/submolts/{submolt_name}/subscribe",
            headers=HEADERS,
            timeout=15
        )
        response.raise_for_status()
        result = {
            "success": True,
            "status_code": response.status_code,
            "message": f"Successfully subscribed to /m/{submolt_name} (Official API)",
            "data": response.json() if response.text else {},
            "submolt": submolt_name
        }
        return result
    except requests.exceptions.HTTPError as e:
        return {
            "success": False,
            "status_code": response.status_code if 'response' in locals() else None,
            "error": f"Subscribe failed (HTTP Error): {str(e)}",
            "submolt": submolt_name,
            "hint": "Check submolt name or API permission (refer to skill.md #Submolts)"
        }
    except Exception as e:
        return {
            "success": False,
            "error": f"Subscribe failed (System Error): {str(e)}",
            "submolt": submolt_name
        }

# -------------------------- 3. upvote_post --------------------------
@tool
def upvote_post(post_id: str = "47ff50f3-8255-4dee-87f4-2c3637c7351c") -> dict:
    """
    Upvote a Moltbook post (Official API: /posts/{id}/upvote)
    Args:
        post_id: Target post ID (default: homework specified ID)
    Returns:
        dict: Upvote result with success status
    """
    try:
        response = requests.post(
            url=f"{BASE_URL}/posts/{post_id}/upvote",
            headers=HEADERS,
            timeout=15
        )
        response.raise_for_status()
        result = {
            "success": True,
            "status_code": response.status_code,
            "message": f"Post {post_id} upvoted successfully (Official API)",
            "data": response.json() if response.text else {},
            "post_id": post_id
        }
        return result
    except requests.exceptions.HTTPError as e:
        return {
            "success": False,
            "status_code": response.status_code if 'response' in locals() else None,
            "error": f"Upvote failed (HTTP Error): {str(e)}",
            "post_id": post_id,
            "hint": "Check post ID or upvote limit (refer to skill.md #Posts)"
        }
    except Exception as e:
        return {
            "success": False,
            "error": f"Upvote failed (System Error): {str(e)}",
            "post_id": post_id
        }

# -------------------------- 4. comment_post --------------------------
@tool
def comment_post(
    post_id: str = "47ff50f3-8255-4dee-87f4-2c3637c7351c",
    content: str = "This content is very helpful for FTEC5660 homework!"
) -> dict:
    """
    Comment on a Moltbook post (Official API: /posts/{id}/comments)
    Args:
        post_id: Target post ID (default: homework specified ID)
        content: Comment content (default: reasonable English comment for FTEC5660)
    Returns:
        dict: Comment result with success status
    """
    payload = {"content": content}
    try:
        response = requests.post(
            url=f"{BASE_URL}/posts/{post_id}/comments",
            headers=HEADERS,
            json=payload,
            timeout=15
        )
        response.raise_for_status()
        result = {
            "success": True,
            "status_code": response.status_code,
            "message": f"Comment posted to post {post_id} (Official API)",
            "data": response.json() if response.text else {},
            "post_id": post_id,
            "comment": content
        }
        return result
    except requests.exceptions.HTTPError as e:
        return {
            "success": False,
            "status_code": response.status_code if 'response' in locals() else None,
            "error": f"Comment failed (HTTP Error): {str(e)}",
            "post_id": post_id,
            "hint": "Check post ID or comment content length (refer to skill.md #Comments)"
        }
    except Exception as e:
        return {
            "success": False,
            "error": f"Comment failed (System Error): {str(e)}",
            "post_id": post_id
        }

# -------------------------- 5. basic tools --------------------------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """
    Fetch Moltbook feed (Official API: /feed)
    官方基础工具：获取首页信息流（文档指定参数：sort/new/hot/top, limit）
    Args:
        sort: Feed sort type (new/hot/top, default: new)
        limit: Number of posts to fetch (default: 10)
    Returns:
        dict: Feed data or error info
    """
    valid_sorts = ["new", "hot", "top"]
    if sort not in valid_sorts:
        return {"success": False, "error": f"Invalid sort type: {sort}, valid: {valid_sorts}"}

    try:
        response = requests.get(
            url=f"{BASE_URL}/feed",
            headers=HEADERS,
            params={"sort": sort, "limit": limit},
            timeout=15
        )
        response.raise_for_status()
        return {
            "success": True,
            "data": response.json() if response.text else {},
            "sort": sort,
            "limit": limit
        }
    except Exception as e:
        return {"success": False, "error": f"Fetch feed failed: {str(e)}"}

@tool
def search_moltbook(query: str, type: str = "submolt") -> dict:
    """
    Semantic search Moltbook (Official API: /search)
    Args:
        query: Search keyword (e.g., "ftec5660")
        type: Search type (all/posts/submolts/agents/comments, default: submolt)
    Returns:
        dict: Search results or error info
    """
    valid_types = ["all", "posts", "submolts", "agents", "comments"]
    if type not in valid_types:
        return {"success": False, "error": f"Invalid type: {type}, valid: {valid_types}"}

    try:
        response = requests.get(
            url=f"{BASE_URL}/search",
            headers=HEADERS,
            params={"q": query, "type": type},
            timeout=15
        )
        response.raise_for_status()
        return {
            "success": True,
            "data": response.json() if response.text else {},
            "query": query,
            "type": type
        }
    except Exception as e:
        return {"success": False, "error": f"Search failed: {str(e)}", "query": query, "type": type}

@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """
    Create a new text post (Official API: /posts)

    Args:
        submolt: Target submolt name (e.g., "ftec5660")
        title: Post title (max 200 chars, official limit)
        content: Post content (max 10000 chars, official limit)
    Returns:
        dict: Post creation result or error info
    """

    if len(title) > 200:
        return {"success": False, "error": "Title exceeds 200 chars (official limit)"}
    if len(content) > 10000:
        return {"success": False, "error": "Content exceeds 10000 chars (official limit)"}

    payload = {"submolt": submolt, "title": title, "content": content}
    try:
        response = requests.post(
            url=f"{BASE_URL}/posts",
            headers=HEADERS,
            json=payload,
            timeout=15
        )
        response.raise_for_status()
        return {
            "success": True,
            "status_code": response.status_code,
            "message": "Post created successfully (Official API)",
            "data": response.json() if response.text else {},
            "submolt": submolt
        }
    except Exception as e:
        return {"success": False, "error": f"Create post failed: {str(e)}", "submolt": submolt}

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime, UTC
from typing import Any


TARGET_SUBMOLT = "ftec5660"
TARGET_POST_ID = "47ff50f3-8255-4dee-87f4-2c3637c7351c"
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')

SYSTEM_PROMPT = """
You are a Moltbook AI agent for FTEC5660 homework.

Your core purpose (MUST follow in order):
1. First verify the Moltbook API key validity (call moltbook_verify_auth)
2. Subscribe to the /m/ftec5660 submolt (call subscribe_submolt with submolt_name=ftec5660)
3. Upvote the specified post (call upvote_post with post_id=47ff50f3-8255-4dee-87f4-2c3637c7351c)
4. Post a reasonable English comment to the above specified post (call comment_post with post_id=47ff50f3-8255-4dee-87f4-2c3637c7351c, content="Useful content for FTEC5660 homework!")
5. After completing all tasks, stop all tool calls and return "FTEC5660 homework tasks completed: auth success + subscribe ftec5660 + upvote post + comment post"

General behavior rules (MUST obey):
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits
- Before posting, ALWAYS search Moltbook to avoid duplication
- Only comment if you add new insight
- Upvote only genuinely useful content
- If uncertain, do nothing
- Prefer short, clear, professional language
- If a human gives an instruction, obey it exactly
- Stop all operations immediately if any step fails, and return the error message

Available tools:
- moltbook_verify_auth: Verify Moltbook API key validity (FIRST STEP)
- subscribe_submolt: Subscribe to a Moltbook submolt
- get_feed: Fetch Moltbook feed
- search_moltbook: Semantic search Moltbook posts, comments, agents
- create_post: Create a new text post
- comment_post: Comment on a post
- upvote_post: Upvote a post
"""

def log(section: str, message: str):
    ts = datetime.now(UTC).strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop (FTEC5660 homework mode)")
    log("INIT", f"Target submolt: {TARGET_SUBMOLT} | Target post ID: {TARGET_POST_ID}")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
        vertexai=True,
    )

    tools = [
        moltbook_verify_auth,
        subscribe_submolt,
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        homework_instruction = f"""
        Complete FTEC5660 homework tasks in order:
        1. Verify Moltbook API key (call moltbook_verify_auth)
        2. Subscribe to /m/{TARGET_SUBMOLT} (call subscribe_submolt)
        3. Upvote post {TARGET_POST_ID} (call upvote_post)
        4. Comment post {TARGET_POST_ID} with "Useful content for FTEC5660 homework!" (call comment_post)
        """
        history.append(("human", homework_instruction))
        log("HOMEWORK", "Executing FTEC5660 core tasks (autonomous mode)")

    # Main agent loop
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # STOP CONDITION
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            if "FTEC5660 homework tasks completed" in response.content:
                log("HOMEWORK", "✅ All FTEC5660 homework tasks completed successfully!")
            return response.content

        # TOOL EXECUTION
        task_failed = False
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
                if isinstance(result, dict) and result.get("success") is False:
                    status = "error"
                    task_failed = True
                    log("TOOL.ERROR", f"{tool_name} execution failed: {result.get('error')}")
            except Exception as e:
                result = {"error": str(e), "success": False}
                status = "error"
                task_failed = True

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

            if task_failed:
                log("HOMEWORK", f"❌ FTEC5660 task failed at {tool_name}, stopping agent...")
                break

        if task_failed:
            log("STOP", "Task failed, agent stopped early")
            return f"FTEC5660 homework failed: {tool_name} error - {result.get('error')}"

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # MAX TURNS REACHED
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns (FTEC5660 homework incomplete)."

def run_ft5660_homework():
    """
    FTEC5660 Moltbook homework
    """
    log("HOMEWORK", "=====================================")
    log("HOMEWORK", "Starting FTEC5660 Moltbook Homework")
    log("HOMEWORK", "=====================================")
    final_result = moltbook_agent_loop()
    log("HOMEWORK", "=====================================")
    log("HOMEWORK", "FTEC5660 Homework Final Result:")
    log("HOMEWORK", final_result)
    log("HOMEWORK", "=====================================")
    return final_result

if __name__ == "__main__":
    run_ft5660_homework()

[17:44:54] [HOMEWORK] =====================================
[17:44:54] [HOMEWORK] Starting FTEC5660 Moltbook Homework
[17:44:54] [HOMEWORK] =====================================
[17:44:54] [INIT] Starting Moltbook agent loop (FTEC5660 homework mode)
[17:44:54] [INIT] Target submolt: ftec5660 | Target post ID: 47ff50f3-8255-4dee-87f4-2c3637c7351c
[17:44:54] [HOMEWORK] Executing FTEC5660 core tasks (autonomous mode)
[17:44:54] [TURN] Turn 1/8 started
[17:44:56] [LLM] Model responded
[17:44:56] [LLM.CONTENT] <empty>
[17:44:56] [LLM.TOOL_CALLS] [
  {
    "name": "moltbook_verify_auth",
    "args": {},
    "id": "23aa8a39-40cd-4fc5-a750-d1d470101917",
    "type": "tool_call"
  }
]
[17:44:56] [TOOL] [1] Calling `moltbook_verify_auth`
[17:44:56] [TOOL.ARGS] {}
[DEBUG] Auth Response Status: 404
[DEBUG] Auth Raw Content: {"statusCode":404,"message":"Cannot GET /api/v1/me","timestamp":"2026-02-26T17:44:56.842Z","path":"/api/v1/me","error":"Not Found"}
[17:44:56] [TOOL.RESULT] moltbook_verify_aut